### 필요한 Library 설치

In [9]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from tqdm.notebook import tqdm
import time
import os

### 케이스 노트의 url 선언

In [3]:
url = 'https://casenote.kr/search/?q=%ED%8F%AD%EB%A0%A5'
resp = requests.get(url)
soup = bs(resp.content, 'lxml')

item = soup.select('div.title a') # item에 원하는 판례의 url을 가지고 오기 위해 판례별 고유 title(id)를 가지고 온다.
for i in item:
    print(i['href']) # 고유의 title 내부의 href를 가지고 온다.

/%EB%8C%80%EB%B2%95%EC%9B%90/2020%EB%8F%841153
/%EC%88%98%EC%9B%90%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90/2016%EB%85%B82190
/%EA%B4%91%EC%A3%BC%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90/2020%EA%B5%AC%ED%95%A915277
/%EC%88%98%EC%9B%90%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90/2015%EB%85%B86073
/%EB%8C%80%EB%B2%95%EC%9B%90/2012%EB%8F%845090
/%EB%8C%80%EA%B5%AC%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90/2012%EB%85%B8653
/%EC%B0%BD%EC%9B%90%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90/2018%EA%B5%AC%EB%8B%A812153
/%EB%8C%80%EA%B5%AC%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90/2018%EA%B5%AC%ED%95%A921875
/%EC%B6%98%EC%B2%9C%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90/2014%EA%B3%A0%ED%95%A942
/%EC%9A%B8%EC%82%B0%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90/2016%EA%B3%A0%ED%95%A9423


### 크롤링을 위한 함수 구현

In [5]:
## 원하는 주제의 각 판례들의url을 가지고 오기 위한 함수
def get_url(topic):
    url_list = []
    for i in tqdm(range(1,21)): # casenote 페이지는 각 주제당 무료버전의 경우 최대 21page까지 제공한다.
        url = f'https://casenote.kr/search/?q={topic}&page={i}' # 각 주제의 url을 만들어준다.
        resp = requests.get(url) # url을 통해 html을 가져온다.
        soup = bs(resp.content, 'lxml') # html을 파싱한다.
        item = soup.select('div.title a') # 각 판례의 고유 title을 가지고 온다.
        for i in item: # 각 판례의 고유 title 내부의 href 즉 각 주소를 가지고 온다.
            url_list.append('https://casenote.kr'+i['href']) # 각 판례의 url을 url_list에 저장한다.


    return url_list # 각 판례의 url을 가지고 있는 url_list를 반환한다.

In [6]:
## 각 판례의 이유, 요지, 참조조문, 참조판례를 가지고 오는 함수
def get_info(url_list, topic): #데이터 프레임 반환
    reason = []
    summary = []
    reflaw = []
    refcases = []
    for i in tqdm(url_list):
        resp = requests.get(i) # 각 판례의 url을 통해 html을 가져온다.
        soup = bs(resp.content, 'lxml') # html을 파싱한다.
        if soup.select('div.reason'):  # 각 판례의 이유를 가지고 온다.
            for z in soup.select('div.reason'):
                reason.append(z.text) # 각 판례의 이유를 reason에 저장한다.
        else:
            reason.append('no data') # 이유가 없는 경우 no data를 저장한다.
        if soup.select('div.summary'): # 각 판례의 요지를 가지고 온다.
            for j in soup.select('div.summary'):
                summary.append(j.text) # 각 판례의 요지를 summary에 저장한다.
        else:
            summary.append('no data') # 요지가 없는 경우 no data를 저장한다.
        if soup.select('div.reflaws'): # 각 판례의 참조조문을 가지고 온다.
            for x in soup.select('div.reflaws'):
                reflaw.append(x.text) # 각 판례의 참조조문을 reflaw에 저장한다.
        else:
            reflaw.append('no data')
        if soup.select('div.refcases'): # 각 판례의 참조판례를 가지고 온다.
            for y in soup.select('div.refcases'):
                refcases.append(y.text) # 각 판례의 참조판례를 refcases에 저장한다.
        else:
            refcases.append('nodata')
        time.sleep(3)
    print(len(reason), len(reflaw), len(summary), len(refcases)) # 각 판례의 이유, 요지, 참조조문, 참조판례의 개수를 출력한다.
    if len(reason) > 200:
        reason = reason[:200]
    if len(reflaw) >200:
        reflaw = reflaw[:200]
    df = pd.DataFrame({"이유": reason, "참조 조문": reflaw, "요지" :summary, "참조 판례" : refcases}) # 각 판례의 이유, 요지, 참조조문, 참조판례를 데이터 프레임으로 만든다.

    df.to_csv(f"{topic}결과.csv", index = False)
    return df

In [7]:
## 각 주제의 판례를 크롤링하는 함수
# 최종적인 구조
def final(topic):
    url_list = get_url(topic)
    df  = get_info(url_list, topic)
    print(f"{topic} done!")
    return df

In [10]:
file = []
for i in os.listdir():
    if 'csv' in i:
        file.append(i)

In [ ]:
for file in file:
    temp_df = pd.read_csv(file)
    temp_df.to_csv('')